In [1]:
!nvidia-smi

Wed Nov 20 17:40:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              8W /   35W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pwd

'd:\\Projects\\DETECT_TRACK_AND_COUNT_USING_YOLOv8'

Upload Videos

In [3]:
SOURCE_PATH_1='D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/1900-151662242_small.mp4'

print(SOURCE_PATH_1)

D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/1900-151662242_small.mp4


In [4]:
SOURCE_PATH_2 ='D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/2165-155327596_small.mp4'

In [5]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [6]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.34  Python-3.8.20 torch-2.4.1+cpu CPU (AMD Ryzen 9 5980HS with Radeon Graphics)
Setup complete  (16 CPUs, 31.4 GB RAM, 14.5/244.1 GB disk)


In [7]:
import os
HOME = os.getcwd()
print(HOME)

d:\Projects\DETECT_TRACK_AND_COUNT_USING_YOLOv8


In [ ]:
%pip install lapx
%pip install -q loguru
%pip install -q thop

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
# sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
%pip install -q cython_bbox
%pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
%pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)


In [ ]:
from IPython import display
display.clear_output()

import sys
sys.path.append(f"{HOME}/ByteTrack")

import yolox
print("yolox.__version__:", yolox.__version__)

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

install RoboFlow Supervision

In [ ]:
%pip install supervision==0.1.0

In [ ]:
from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

Load pretrain model

In [ ]:
MODEL = "yolov8x.pt "

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

In [ ]:
%cd

In [ ]:
# conda install -c conda-forge ultralytics

In [ ]:
# print(SOURCE_PATH_1)

!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_PATH_1}

Detection on frame of the video

In [ ]:
from supervision.video.source import get_video_frames_generator
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.draw.color import ColorPalette

In [ ]:
CLASS_NAMES_DICT = model.model.names
CLASS_NAMES_DICT

In [ ]:
#dict mapping class_id to class_name
CLASS_ID = [2,3,5,7]

Predict on Single frame

In [ ]:
#create frame generator
generator = get_video_frames_generator(SOURCE_PATH_1)
#create instance of BoxAnnotator
box_annotator = BoxAnnotator(color = ColorPalette(), thickness = 1, text_thickness=1, text_scale=0.9)
#acquire first video frame
iterator = iter(generator)
frame = next(iterator)

#model prediction on single frame and conversion to supervision Detections
results = model(frame)
detections = Detections(
    xyxy = results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)

#format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]

#annotate and display frame
frame= box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (16,16))

Detect and track whole video

In [ ]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
SOURCE_PATH_2 = "D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/2165-155327596_small.mp4"

In [ ]:
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator
from supervision.video.sink import VideoSink
import numpy as np

In [ ]:
# settings
LINE_START = Point(50, 500)
LINE_END = Point(960-50, 500)

TARGET_VIDEO_PATH = "vehicle-counting-result.mp4"

In [ ]:
VideoInfo.from_video_path(SOURCE_PATH_2)

In [ ]:
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
# !pip install ipywidgets

In [ ]:
from tqdm.notebook import tqdm


# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_PATH_2)
# create frame generator
generator = get_video_frames_generator(SOURCE_PATH_2)
# create LineCounter instance
line_counter = LineCounter(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # format custom labels
        labels = [
            f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        # updating line counter
        line_counter.update(detections=detections)
        # annotate and display frame
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        line_annotator.annotate(frame=frame, line_counter=line_counter)
        sink.write_frame(frame)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html